In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from torchvision.transforms import Resize, InterpolationMode, ToPILImage, RandomCrop
from torchmetrics import JaccardIndex, Precision, Recall, F1Score
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from src.callbacks.SaveRandomImagesCallback import SaveRandomImagesCallback
from src.callbacks.SaveTestPredsMulticlass import SaveTestPredsMulticlass
import segmentation_models_pytorch as smp

from src.evaluation.evaluate_result import evaluate_result
from src.datasets.AerialDroneSemanticSegmentationDataset import (
    AerialDroneSemanticSegmentationDataset,
)

## Prepare environment

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if you get some cryptic CUDA error, set device to "cpu" and try again

# DEBUG
# device = torch.device("cpu")
print(device)

In [ ]:
VAL_SIZE = 0.2
BATCH_SIZE = 1
SEED = 42
AERIAL_DRONE_DATASET_PATH = (
    "data/AerialDroneSemanticSegmentationDataset/dataset/semantic_drone_dataset"
)
SAVE_VAL_DIR = "outputs/AerialDrone/val"
SAVE_TEST_DIR = "outputs/AerialDrone/test"

# INRIA_DATASET_PATH = "data/TestSubsets/INRIAAerialImageLabellingDataset"  # laptop
IMAGE_SIZE = 576  # home PC

IMAGE_HEIGHT = 608
IMAGE_WIDTH = 416

In [ ]:
labeled_dataset = AerialDroneSemanticSegmentationDataset(
    AERIAL_DRONE_DATASET_PATH,
    transforms=[
        RandomCrop((IMAGE_HEIGHT, IMAGE_WIDTH)),
    ],
)
print(len(labeled_dataset))

## Data preparation

In [ ]:
sanity_check_loader = DataLoader(labeled_dataset, batch_size=BATCH_SIZE, shuffle=False)

### Train, validation and test split

In [ ]:
train_size = int(0.7 * len(labeled_dataset))
val_size = int(0.2 * len(labeled_dataset))
test_size = len(labeled_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(
    labeled_dataset, [train_size, val_size, test_size]
)
print(len(train_dataset), len(val_dataset), len(test_dataset))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

In [ ]:
to_pil_transform = ToPILImage()

In [ ]:
if BATCH_SIZE == 1:
    img = to_pil_transform(images.squeeze())

In [ ]:
# img.show()

In [ ]:
if BATCH_SIZE == 1:
    msk = to_pil_transform(masks.squeeze())

In [ ]:
# msk.show()

In [ ]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

In [ ]:
to_pil_transform = ToPILImage()

In [ ]:
img = to_pil_transform(images.squeeze())

In [ ]:
# img.show()

In [ ]:
msk = to_pil_transform(masks.squeeze())

In [ ]:
# msk.show()

# Training module

In [ ]:
class SegmentationModel(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super(SegmentationModel, self).__init__()
        
        self.model = model
        self.learning_rate = learning_rate

        self.criterion = smp.losses.JaccardLoss(mode="multiclass", from_logits=False)
        self.train_iou = JaccardIndex(num_classes=23, task="multiclass")
        self.val_iou = JaccardIndex(num_classes=23, task="multiclass")

        # self.save_hyperparameters()

    def forward(self, x):
        output = self.model(x.to(device))
        return output

    def training_step(self, batch, batch_idx):
        images, masks = batch
        preds = self(images)
        loss = self.criterion(preds, masks.squeeze(1).long())

        self.log("train_loss", loss, on_epoch=True, on_step=True)
        self.log("train_iou", self.train_iou(preds, masks.squeeze(1)), on_epoch=True, on_step=True)

        return loss

    def validation_step(self, batch, batch_idx):
        images, masks = batch
        preds = self(images)
        loss = self.criterion(preds, masks.squeeze(1).long())

        self.log("val_loss", loss, on_epoch=True, on_step=True)
        self.log("val_iou", self.val_iou(preds, masks.squeeze(1)), on_epoch=True, on_step=True)
        
        return loss

    def test_step(self, batch, batch_idx):
        # just here to activate the test_epoch_end
        # callback SaveTestPreds starts on_test_epoch_end 
        pass

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

### Model selection

In [ ]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=23,  # model output channels (number of classes in your dataset)
    activation="softmax",
).to(device)

In [ ]:
# model = smp.UnetPlusPlus(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,  # model output channels (number of classes in your dataset)
# ).to(device)

In [ ]:
# model = smp.DeepLabV3(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,
# ).to(device)

In [ ]:
# model = smp.DeepLabV3Plus(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,
# ).to(device)

In [ ]:
segmentation_model = SegmentationModel(model)

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    monitor="val_loss", save_top_k=-1, mode="min"
)

In [ ]:
logger = CSVLogger("lightning_logs", name="aerial_drone_segmentation_model")

In [ ]:
save_callback = SaveRandomImagesCallback(save_dir=SAVE_VAL_DIR)

# Dubai test dataset is also labeled
# so we can use different logic for test set evaluation
save_test_preds_callback = SaveTestPredsMulticlass(save_dir=SAVE_TEST_DIR)

early_stopping_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5)

In [ ]:
trainer = pl.Trainer(
    max_epochs=1,
    log_every_n_steps=10,
    callbacks=[
        model_checkpoint_callback,
        save_callback,
        save_test_preds_callback,
        early_stopping_callback,
    ],
    logger=logger,
)

# DEBUG
# trainer = pl.Trainer(
#     max_epochs=1,
#     callbacks=[model_checkpoint_callback, save_callback],
#     logger=logger,
#     accelerator="cpu"
#     )

In [ ]:
trainer.fit(
    segmentation_model, train_dataloaders=train_loader, val_dataloaders=val_loader
)

In [ ]:
trainer.test(ckpt_path="best", dataloaders=test_loader)